In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
class NCFDataset(Dataset):
    def __init__(self, interactions, emotions, review_embeddings):
        self.users, self.items = interactions.nonzero()
        self.ratings = interactions[self.users, self.items].A1
        self.emotions = emotions[self.items].toarray()  # Convert to dense array
        self.review_embeddings = review_embeddings[self.items].toarray()  # Convert to dense array

    def __len__(self):
        return len(self.users)
    
    def __getitem__(self, idx):
        return (self.users[idx], self.items[idx], self.emotions[idx], 
                self.review_embeddings[idx], self.ratings[idx])

class NCF(nn.Module):
    def __init__(self, num_users, num_items, num_emotions, review_embedding_dim, 
                 embedding_dim=64, mlp_dims=[256, 128, 64], dropout=0.2):
        super(NCF, self).__init__()
        
        # Embedding layers
        self.user_embedding_mf = nn.Embedding(num_users, embedding_dim)
        self.item_embedding_mf = nn.Embedding(num_items, embedding_dim)
        self.user_embedding_mlp = nn.Embedding(num_users, embedding_dim)
        self.item_embedding_mlp = nn.Embedding(num_items, embedding_dim)
        self.emotion_embedding = nn.Embedding(num_emotions, embedding_dim)
        
        # MF layer
        self.mf_output = embedding_dim
        
        # MLP layers
        self.mlp = nn.ModuleList()
        input_dim = embedding_dim * 3 + review_embedding_dim  # user + item + emotion + review
        mlp_dims = [input_dim] + mlp_dims
        for i in range(len(mlp_dims) - 1):
            self.mlp.append(nn.Linear(mlp_dims[i], mlp_dims[i+1]))
            self.mlp.append(nn.ReLU())
            self.mlp.append(nn.BatchNorm1d(mlp_dims[i+1]))
            self.mlp.append(nn.Dropout(dropout))
        
        # Final prediction layer
        self.final = nn.Linear(self.mf_output + mlp_dims[-1], 1)
        
    def forward(self, user_indices, item_indices, emotion_indices, review_embeddings):
        # MF component
        user_embedding_mf = self.user_embedding_mf(user_indices)
        item_embedding_mf = self.item_embedding_mf(item_indices)
        mf_vector = torch.mul(user_embedding_mf, item_embedding_mf)
        
        # MLP component
        user_embedding_mlp = self.user_embedding_mlp(user_indices)
        item_embedding_mlp = self.item_embedding_mlp(item_indices)
        emotion_embedding = self.emotion_embedding(emotion_indices)
        emotion_embedding = self.emotion_embedding(emotion_indices).mean(dim=1)  # Adjusted for mean over emotions
        mlp_vector = torch.cat([user_embedding_mlp, item_embedding_mlp, emotion_embedding, review_embeddings], dim=-1)
 
        for layer in self.mlp:
            mlp_vector = layer(mlp_vector)
        
        # Combine MF and MLP
        combined = torch.cat([mf_vector, mlp_vector], dim=-1)
        
        # Final prediction
        prediction = self.final(combined)
        
        return prediction.squeeze()

In [2]:
# Get the data split
test_dataset = torch.load("/Users/User/Downloads/test_dataset.pth/test_dataset.pth")

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import mean_squared_error
# Load the model state dictionaries
state_dict1 = torch.load('/Users/User/Downloads/NCF_model.pth', map_location=torch.device('cpu'))
state_dict2 = torch.load('/Users/User/Downloads/NCF_model.pth', map_location=torch.device('cpu'))

# Recreate the model architecture
def create_model(state_dict):
    num_users = state_dict['user_embedding_mf.weight'].shape[0]
    num_items = state_dict['item_embedding_mf.weight'].shape[0]
    num_emotions = state_dict['emotion_embedding.weight'].shape[0]
    embedding_dim = state_dict['user_embedding_mf.weight'].shape[1]  # This should be 32
    review_embedding_dim = state_dict['mlp.0.weight'].shape[1] - 3 * embedding_dim

    # Adjust the NCF class initialization to match the saved model
    model = NCF(num_users, num_items, num_emotions, review_embedding_dim, embedding_dim=embedding_dim)
    model.load_state_dict(state_dict)
    model.eval()
    return model

# Create two model instances
model1 = create_model(state_dict1)
model2 = create_model(state_dict2)

# Create a DataLoader for the test dataset
batch_size = 64  # Adjust as needed
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Function to make predictions
def make_predictions(model, loader):
    predictions = []
    with torch.no_grad():
        for batch in loader:
            user_indices, item_indices, emotions, review_embeddings, _ = batch
            # Convert indices to Long (integer) type
            user_indices = user_indices.long()
            item_indices = item_indices.long()
            emotions = emotions.long()
            review_embeddings = review_embeddings.float()
            outputs = model(user_indices, item_indices, emotions, review_embeddings)
            predictions.extend(outputs.cpu().numpy())
    return np.array(predictions)

# Make predictions with both models
predictions1 = make_predictions(model1, test_loader)
predictions2 = make_predictions(model2, test_loader)

# Ensemble the predictions
ensemble_predictions = (predictions1 + predictions2) / 2

# Get the true ratings
true_ratings = []
for batch in test_loader:
    _, _, _, _, ratings = batch
    true_ratings.extend(ratings.cpu().numpy())
true_ratings = np.array(true_ratings)

# Calculate MSE
mse = mean_squared_error(true_ratings, ensemble_predictions)
print(f"Ensemble Model MSE: {mse}")

# Optional: Save the ensemble model
ensemble_model = {
    'model1': model1.state_dict(),
    'model2': model2.state_dict(),
    'weights': [0.5, 0.5]  # Equal weights for both models
}

torch.save(ensemble_model, 'ensemble_ncf_model.pth')

Ensemble Model MSE: 0.02643394988215008
